Link dos datasets

Samba: https://www.kaggle.com/datasets/clovesgtx/brazilian-music-samba-lyrics

Gospel: https://raw.githubusercontent.com/damarals/letras/master/inst/csv/letras.csv

Axe: https://www.kaggle.com/datasets/jorgefjr/brazilian-songs-lyrics

Funk: https://www.kaggle.com/datasets/sleshes/funk-carioca-dataset

Bossa Nova: https://www.kaggle.com/datasets/mcarujo/bossa-nova-lyrics

Rock,Sertanejo, Samba2 = https://github.com/gabriellmd/data-frames-nlp

In [ ]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

In [2]:
bossa = pd.read_csv('datasets/bossa_nova_songs_dataset.csv')
samba2 = pd.read_csv('datasets/musicas_samba.csv')
sertanejo = pd.read_csv('datasets/musicas_sertanejo.csv')
samba = pd.read_csv('datasets/samba_dataset.csv',sep='|')
axe = pd.read_csv('datasets/letras_mus_br_axe.csv')
funk = pd.read_csv('datasets/Dataset_funk_carioca.csv')
rock = pd.read_csv('datasets/musicas_rock.csv') 
gospel = pd.read_csv('datasets/gospel.csv') 

Padronização dos datasets

In [3]:
gospel_ajustado = pd.DataFrame(gospel['letra'])  #OK
gospel_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
rock_ajustado = pd.DataFrame(rock['Lyric'])  #OK
rock_ajustado.rename(columns={'Lyric': 'lyric'}, inplace = True) #OK
samba_ajustado = pd.DataFrame(samba['letra'])  #OK
samba_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
samba2_ajustado = pd.DataFrame(samba2['Lyric'])  #OK
samba2_ajustado.rename(columns={'Lyric': 'lyric'}, inplace = True) #OK
axe_ajustado = pd.DataFrame(axe['letras'])  #OK
axe_ajustado.rename(columns={'letras': 'lyric'}, inplace = True) #OK
funk_ajustado = pd.DataFrame(funk['letra'])  #OK
funk_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
bossa_ajustado = pd.DataFrame(bossa['song_lyrics'])  #OK
bossa_ajustado.rename(columns={'song_lyrics': 'lyric'}, inplace = True) #OK
sertanejo_ajustado = pd.DataFrame(sertanejo['Lyric'])  #OK
sertanejo_ajustado.rename(columns={'Lyric': 'lyric'}, inplace = True) #OK

Criando a coluna correspondete ao genero

In [4]:
bossa_ajustado['gen'] = 'Bossa Nova'
funk_ajustado['gen'] = 'Funk'
gospel_ajustado['gen'] = 'Gospel'
sertanejo_ajustado['gen'] = 'Sertanejo'
axe_ajustado['gen'] = 'Axe'
rock_ajustado['gen'] = 'Rock'
samba_ajustado['gen'] = 'Samba'
samba2_ajustado['gen'] = 'Samba'

In [5]:
datasets = [bossa_ajustado,funk_ajustado,gospel_ajustado,sertanejo_ajustado,axe_ajustado,rock_ajustado,samba_ajustado,samba2_ajustado]

In [6]:
lyrics = pd.concat(datasets)

Dataset final

In [7]:
lyrics.reset_index(drop=True,inplace=True)
lyrics

,lyric,gen
0,"De tudo, ao meu amor serei atento antes E co...",Bossa Nova
1,Era uma casa Muito engraçada Não tinha teto ...,Bossa Nova
2,E por falar em saudade Onde anda você Onde a...,Bossa Nova
3,Filhos... Filhos? Melhor não tê-los! Mas se ...,Bossa Nova
4,É melhor ser alegre que ser triste Alegria é...,Bossa Nova
...,...,...
65762,Tom............:. Introdução.: Dm C#o Gm C7 F ...,Samba
65763,"Resolvi te abandonar te deixar para traz, deci...",Samba
65764,Ela mexe comigo. E o pior que não sabe. Coment...,Samba
65765,O pai me disse que a tradição é lanterna. Vem ...,Samba


## Pré-processamento

In [8]:
def processamento_texto(txt):
    # remover quebras de linha
    txt = txt.replace('\n',' ')
    # remover símbolos de pontuação, resultando em um array de caracteres
    txt = [char for char in txt if char not in string.punctuation]
    # depois, juntar os caracteres em palavras novamente e separá-los em uma lista de tokens
    txt = ''.join(txt).split()
    # por fim, remover as stopwords da lista
    #txt = [word for word in txt if word.lower() not in stopwords.words('portuguese')]
    
    return txt

In [9]:
n = '\nNaiveB:'
r = '\nRandomF:'
svm = '\nSuppotVM:'
naive_bayes_array = np.array([])
random_forest_array = np.array([])
support_vecto_machines_array = np.array([])

In [12]:
lyric_train,lyric_test,gen_train,gen_test = train_test_split(lyrics['lyric'],lyrics['gen'],test_size=0.3)

In [13]:
pipelineSVC = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('svc',SVC())
])

In [14]:
pipelineSVC.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function processamento_texto at 0x000001F7BA5408B0>,
                                 encoding='unicode')),
                ('tfidf', TfidfTransformer()), ('svc', SVC())])

In [ ]:
scoring = {'acc': 'accuracy'}
scores_svm = cross_validate(pipelineSVC,lyrics['lyric'].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)